In [232]:
import requests
import pandas as pd
import numpy as np
import sys
import os
import time
from stravaio import strava_oauth2
from stravaio import StravaIO
import polyline as pl

# Authenticate with the Strava API
You must set up your local environment with the following variables:

`STRAVA_ACCESS_TOKEN`

`STRAVA_CLIENT_ID`

`STRAVA_CLIENT_SECRET`

These can be found at https://www.strava.com/settings/api

(If these environment variables are set, `strava_oauth2` requires no arguments. Alternatively, you may pass the ID and client secret as `strava_oauth2(client_id=[your id], client_secret=[your secret]`.)

In [233]:
# Authenticate with Strava; opens a new tab for interactive authentication.
client = strava_oauth2()

2020-05-28 13:58:54.261 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2020-05-28 13:58:57.713 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: e6c333a76cde9d43f967e9ee0b59c3a919217c3f
2020-05-28 13:58:58.408 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: fda5e02deba99b3ffd94062933fbb0f11f95f008


In [234]:
os.environ["STRAVA_ACCESS_TOKEN"]=client["access_token"]

In [235]:
# If the token is stored as an environment varible it is not neccessary
# to pass it as an input parameters

client = StravaIO()

In [236]:
# Set up a requests Session for calls not already included in `stravaio`
s = requests.Session()
base_url = "https://www.strava.com/api/v3"

In [237]:
access_token = os.environ['STRAVA_ACCESS_TOKEN']

s.headers.update(
    {
    "Authorization": f"Bearer {access_token}"
    }
)

In [238]:
res = s.get(f"{base_url}/athlete/clubs")

stravavalanche = res.json()[
    ["Fatty Fitness International" in club["name"] for club in res.json()].index(True)
]

strv_id = stravavalanche["id"]

The instantiated `StravaIO` object contains a great deal of metadata about your user. Run `client` in the cell below to take a gander.

In [241]:
df = client.get_logged_in_athlete_activities()
ski_days = df[df["type"]=="BackcountrySki"]

Fetched 30, the latests is on 2017-05-01 14:09:10+00:00
Fetched 30, the latests is on 2017-10-25 20:22:10+00:00
Fetched 30, the latests is on 2018-05-14 20:48:36+00:00
Fetched 30, the latests is on 2018-09-20 00:01:00+00:00
Fetched 30, the latests is on 2019-02-17 02:53:00+00:00
Fetched 30, the latests is on 2019-04-21 21:08:04+00:00
Fetched 30, the latests is on 2019-06-26 00:53:54+00:00
Fetched 30, the latests is on 2019-08-10 04:42:58+00:00
Fetched 30, the latests is on 2020-05-21 16:40:01+00:00
Fetched 3, the latests is on 2020-05-25 23:04:43+00:00


TypeError: list indices must be integers or slices, not str

In [ ]:
metadata = ski_days[["id", "athlete", "distance", "elapsed_time", "elev_high", "elev_low", "athlete_count"]]
def get_polyline_from_id(_id):
#     activity = client.get_activity_by_id(_id)
    activity = s.get(f"{base_url}/activities/{_id}").json()
    return pl.decode(activity["map"]["summary_polyline"])
    
metadata["polyline"] = ski_days["id"].apply(lambda x: get_polyline_from_id(x))
metadata.reset_index(inplace=True, drop=True)

In [242]:
metadata

,index,id,athlete,distance,elapsed_time,elev_high,elev_low,athlete_count,polyline
0,62,1261428977,6434174,11452.5,16587,2955.0,2082.0,1,"[(45.81787, -110.8961), (45.81827, -110.90063)..."
1,69,1327173667,6434174,10015.2,6814,2398.5,1632.0,1,"[(45.6091, -110.92453), (45.60694, -110.92616)..."
2,147,2086163817,6434174,2405.6,2006,1858.3,1647.0,1,"[(45.60891, -110.92477), (45.60729, -110.92489..."
3,148,2240851047,6434174,5434.9,2829,1859.7,1643.0,2,"[(45.6092, -110.92431), (45.609, -110.92408), ..."


# Step 2: Get GIS data 